### Домашнее задание 1:

Решить задачу о рюкзаке методом динамического программирования:
Есть рюкзак определенной вместимости и набор предметов с указанным весом и стоимостью. Ваша задача - выбрать оптимальный набор предметов для помещения в рюкзак так, чтобы суммарная стоимость была максимальной, а сумма весов предметов не превышала вместимость рюкзака

1. Постороить матрицу, отражающей максимальную стоимость, которую можно достичь с данным весом и предметами.
2. написать рекурсивную функцию для поиска оптимального решения.

In [2]:
import numpy as np
class Thing: 
    def __init__(self, weight, price):
        self.weight = weight
        self.price = price

    def __str__(self):
        return f"thing: weight = {self.weight}, price = {self.price}"

    def __repr__(self):
        return f"thing: weight = {self.weight}, price = {self.price}"

In [3]:
data = [Thing(2, 3), Thing(4, 1), Thing(6, 4), Thing(1, 3)]
data

[thing: weight = 2, price = 3,
 thing: weight = 4, price = 1,
 thing: weight = 6, price = 4,
 thing: weight = 1, price = 3]

In [4]:
def knapsack(things, capacity):
    N = len(things)
    dp = np.zeros((N, capacity + 1))
    
    things = sorted(things, key=lambda x: x.weight)
    
    for i in range(N):
        for j in range(capacity + 1):
            if i == 0:
                dp[i, j] = things[i].price if j >= things[i].weight else 0
            else:
                dp[i, j] =  max(things[i].price + dp[i-1, j - things[i].weight], dp[i-1, j])  if j >= things[i].weight else dp[i-1, j]

    items = []
    w = capacity
    for i in range(N-1, 0, -1):
        if dp[i, w] != dp[i - 1, w]:
            items.append(things[i])
            w -= things[i].weight
    else:
        if w > 0:
            items.append(things[0])
    return dp.max(), items
    
            
            
    
    






In [5]:
total, things = knapsack(data, 8)
print(f"Общая стоимость: {total}, вещи: ")
print(*things, sep="\n")

Общая стоимость: 7.0, вещи: 
thing: weight = 4, price = 1
thing: weight = 2, price = 3
thing: weight = 1, price = 3


### Домашнее задание 2:

1. Расширить функционал системы управления университетом:
- добавить метод для отчисления студента с курса
- реализовать проверку, чтобы студент не мог быть зачислен на один и тот же курс дважды
- реализовать возможность хранения оценок студентов по каждому курсу и расчета среднего балла

2. Расширить функционал системы управления банковским счетом:
- добавить метод для перевода средств на другой счет
- *реализовать декоратор для проверки наличия достаточных средств перед выполнением операции снятия*

In [6]:
# родительский класс
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def get_details(self):
        return f"Name: {self.name}, Age: {self.age}"

In [7]:
class Student(Person):
   def __init__(self, name, age, student_id, courses=None):
      super().__init__(name, age)
      self.student_id = student_id
      assert courses is None or len(set(courses)) == len(courses), "courses must be unique" 
      self.courses = courses if courses is not None else []

      self.grades = dict()
      for elem in self.courses:
          self.grades[elem] = []

   def enroll(self, course):
      assert not course in self.courses, "The course is already in list"    
      self.courses.append(course)
      self.grades[course] = []

   def get_details(self):
      details = super().get_details()
      return f"{details}, Student ID: {self.student_id}, Courses: {', '.join(self.courses)}"

   def get_grade(self, course, value):
      self.grades[course].append(value)


   def get_average(self, courses=None):
       from numpy import mean
       if courses is None:
           for key, value in self.grades.items():
               print(f"Course: {key}, Avg grade: {mean(value)}")
       else:
           for elem in courses:
               print(f"Course: {elem}, Avg grade: {mean(self.grades[elem])}")
       
    
   def ban_from_course(self, course_name):
      self.courses.remove(course_name)
      print(f"Student {self.get_details()} was deleted from {course_name}")
      del self.grades[course_name]

In [8]:
class Teacher(Person):
   def __init__(self, name, age, teacher_id, subjects=None):
      super().__init__(name, age)
      self.teacher_id = teacher_id
      self.subjects = subjects if subjects is not None else []

   def assign_subject(self, subject):
      self.subjects.append(subject)

   def get_details(self):
      details = super().get_details()
      return f"{details}, Teacher ID: {self.teacher_id}, Subject: {', '.join(self.subjects)}"

In [9]:
def main():
    student = Student(name="Mary", age=20, student_id="F5665")
    student.enroll("Math")
    student.enroll("Physics")
    
    teacher = Teacher(name="N.Smith", age=45, teacher_id='T47655')
    teacher.assign_subject('Math')
    teacher.assign_subject('Computer sciense')


    student.get_grade("Math", 5)
    student.get_grade("Math", 4)
    student.get_grade("Math", 3)
    student.get_grade("Physics", 5)

    student.get_average(["Math"])
    
    
    print("Student:")
    print(student.get_details())

    print("Teacher:")
    print(teacher.get_details())


In [10]:
main()

Course: Math, Avg grade: 4.0
Student:
Name: Mary, Age: 20, Student ID: F5665, Courses: Math, Physics
Teacher:
Name: N.Smith, Age: 45, Teacher ID: T47655, Subject: Math, Computer sciense


### Bank

In [11]:
class BankAccount:
    def __init__(self, owner, initial_balance=0):
        self.owner = owner
        self.__balance = initial_balance  # приватное свойство

    def deposit(self, amount):  # метод для внесения депозита
        self.__balance += amount
        print(f"Вы внесли {amount}руб. Ваш текущий баланс: {self.__balance}руб.")


    @staticmethod
    def check(func):
        def wrapper(self, amount):
            if amount > self.__balance:
                print("Недостаточно средств")
            else:
                return func(self, amount)
        return wrapper

    @check
    def withdraw(self, amount):
        self.__balance -= amount
        print(f"Вы сняли {amount}руб. Ваш текущий баланс: {self.__balance}руб.")

    def chek_balance(self):
        print(f"Ваш текущий баланс: {self.__balance}руб.")

    def to_another_bill(self, name, amount):
        if amount > self.__balance:
            print("Величина перевода выше, чем ваш текущий счет")
        else:
            name.deposit(amount)
            self.__balance -= amount
            print(f"Trunsaction completed")



In [12]:
def main():
    account = BankAccount("Пользователь 1")
    account2 = BankAccount("Пользователь 1")

    while True:
        print("Выберите действие:")
        print("1: Проверить баланс")
        print("2: Внести сумму")
        print("3: Снять сумму")
        print("4: Перевод суммы со счета на другой счет")
        print("5: Выход")

        choice = input("> ")

        if choice == "1":
            account.chek_balance()

        elif choice == "2":
            amount = float(input("Введите сумму для внесения: "))
            account.deposit(amount)

        elif choice == "3":
            amount = float(input("Введите сумму для снятия: "))
            account.withdraw(amount)

        elif choice == "4":
            amount = float(input("Введите сумму для перевода на accaunt2: "))
            
            account.to_another_bill(account2, amount)
            
        elif choice == "5":
            print("До свидания!")
            break

        else:
            print("Попробуйте еще раз.")


if __name__ == "__main__":
  main()

Выберите действие:
1: Проверить баланс
2: Внести сумму
3: Снять сумму
4: Перевод суммы со счета на другой счет
5: Выход
До свидания!
